In [1]:
import pandas as pd
import re # para expresiones regulares
import emoji #convertir emoticones a texto
import nltk #tokenizar y stopwords
from nltk.corpus import stopwords#importamos el corpus de stopwords de NLTK
import stanza #lematizar texto en español
import matplotlib.pyplot as plt# presentar las graficas
# especificamos los procesadores, el lenguaje y con la condicion de que el texto ya esta tokenizado
nlp = stanza.Pipeline(lang='es', processors='tokenize,lemma', tokenize_pretokenized=True) #lematizacion
import pickle

C:\Users\Usuario iTC\AppData\Roaming\Python\Python38\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-08-18 21:20:14 WARNING: Language es package default expects mwt, which has been added
2022-08-18 21:20:14 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| lemma     | ancora  |

2022-08-18 21:20:14 INFO: Use device: cpu
2022-08-18 21:20:14 INFO: Loading: tokenize
2022-08-18 21:20:14 INFO: Loading: mwt
2022-08-18 21:20:14 INFO: Loading: lemma
2022-08-18 21:20:14 INFO: Done loading processors!


Se sigue el mismo procedimiento que se realizó para el preprocesamiento de datos extraídos para el entrenamiento "Carpeta -> 2.Preprocesamiento", con excepción de la limpieza manual para que se pueda usar con otros datos nuevos directamente sin necesidad de realizar otros procedimientos manuales.

### Funciones

In [2]:
#Eliminamos Tweets que son REPLIES
def eliminar_tweet_replies(dataset):
    dataset['length'] = dataset.reply_to.str.len()
    dataset = dataset[dataset.length < 3] 

In [3]:
def eliminar_repetidos(dataset):
    dataset.drop_duplicates(subset=['username','tweet'], inplace=True)
    
def eliminar_atributos_innecesarios(dataset):
    dataset.drop(['conversation_id', 'created_at', 'timezone', 'user_id', 'username', 'name', 'place', 'source', 'user_rt_id', 'user_rt', 'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src', 'trans_dest', 'language', 'mentions', 'urls', 'photos', 'replies_count', 'retweets_count', 'likes_count', 'hashtags', 'cashtags', 'link', 'retweet', 'quote_url', 'video', 'thumbnail', 'near','length'], axis = 1, inplace = True)

In [4]:
#Palabras que no son tomadas en cuenta en la eliminacion de caracteres repetidos
palabras_excluidas=['facebook','mood','boomerang', 'boomer', 'tattoo', 'cool','feeling',
                   'descoord','tweet','desee','cree','lee'] 
#eliminar menciones y hashtags
def limpiar_texto(texto):
    texto = re.sub(r'@[A-Za-z0-9_]+', '', texto) #Remueve menciones
    texto = re.sub(r"[#]+", '', texto) #Remueve hashtags
    #eliminamos caracteres repetidos
    res = any(bool(True if palabra in texto else False) for palabra in palabras_excluidas)
    if not res: #no hay exclusiones
        texto = re.sub(r'(a|e|i|o|u|A|E|I|O|U)\1+', r'\1', texto) # vocales repetidas
    return texto

In [5]:
#eliminamos simbolos especiales, numeros y espacios repetidos
def limpiar_signos(texto):
    texto = re.sub(r'[\“\”\¨‘’,;.…:¡!¡°«»《》•¿?@#$%&[\](){}<>~=+\-–—*/|\\_^`"\']', '', texto) #eliminando simbolos
    texto = re.sub('\d', '', texto) #eliminando numeros presentes en el texto
    texto = re.sub(r'(\s)\1+', r'\1', texto)  #elimina espacios extra en el texto
    return texto

In [6]:
#conversion de emoticones a texto en idioma español
def convertir_emojis(texto):
    texto=emoji.demojize(texto, language='es')  # 💔 --> :corazón_roto:
    return texto

def depurar_emojis(texto):
    texto = re.sub(r'([:][A-Za-z_]+[:])\1+', r'\1', texto)  #eliminar emojis repetidos
    texto = re.sub(r' ?([:][A-Za-z_]+[:]) ?', r' \1 ', texto) #separar emojis unidos
    texto = re.sub(r'(\s)\1+', r'\1', texto)  #eliminar espacios excesivos generados en la separacion de emojis
    texto = re.sub(r'[:]', '', texto)  #elim. los ":" del formato de emoji a texto :cara_triste:-->cara_triste
    return texto

In [7]:
#ELIMINAR TILDES
def quitar_tilde(s):
    reemplazo = (   #se reemplazan las vocales con tilde por su equivalente sin tilde
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
        ("ü", "u"),
    )
    for a, b in reemplazo:
        s = s.replace(a, b)
    return s

In [8]:
def texto_a_minusculas(dataset):
    #Se convierten todo el texto de los tweets a minúsculas
    dataset['tweet_preprocesado'] = dataset['tweet_preprocesado'].str.lower()

In [9]:
#TOKENIZACION
from nltk.tokenize import word_tokenize #divide una oración en tokens o palabras
def tokenizar(texto):
    texto=word_tokenize(texto)
    return texto

In [10]:
#STOPWORDS
stopwords_excluidas={'No', 'Nada', 'Ni', 'me', 'mi'}

# se obtienen la lista de stopwords en español que tiene NLTK
stop_words = set(stopwords.words('spanish'))  #recupera las palabras vacías en español
stop_words.update(("q","xq","pq", "porq","x","d","i","pa")) #agregamos nuevas stopwords comunmente usadas
#print(sorted(stop_words))
def filtrar_stopwords(texto):
    #quitamos las palabras que estan excluidas de la lista de stopwords
    for x in stopwords_excluidas:
        if x.lower() in stop_words:
            stop_words.remove(x.lower())
    #Quitamos tildes a las Stopwords (al ser en español)
    stop_words_tilde=[]
    for x in stop_words:
        x=quitar_tilde(x) #usamos el metodo creado anteriormente para quitar las tildes 
        stop_words_tilde.append(x)
    #se toma el texto tokenizado usando el metodo anteriormente creado  para tokenizar
    word_tokens = tokenizar(texto) 
    filtered_sentence = []
    for w in word_tokens: # se filtran las stopwords
        if w not in stop_words_tilde:
            filtered_sentence.append(w)
    return filtered_sentence

In [11]:
#Lematizacion
def lematizar_texto(texto):
    doc = nlp([texto])
    textLema = []
    for sent in doc.sentences: 
         for word in sent.words:
            textLema.append(quitar_tilde(word.lemma)) # texto aplicado la lematizacion -quitamos tildes
    return textLema

## Preprocesamiento de los Tweets del 2019

Importamos dataset extraído de los tweets del 2019

In [12]:
tweets_2019_df = pd.DataFrame(pd.read_excel("Tweets_2019.xlsx"))

Aplicamos todas las funciones de procesamiento a los tweets de forma directa

In [13]:
#Aplicamos todas las funciones anteriores en una sola
def limpiar_tweets(dataset):
    eliminar_tweet_replies(dataset)
    eliminar_repetidos(dataset)
    eliminar_atributos_innecesarios(dataset)
    dataset['tweet_preprocesado'] = dataset['tweet'].apply(limpiar_texto)
    dataset['tweet_preprocesado'] = dataset['tweet_preprocesado'].apply(limpiar_signos)
    dataset['tweet_preprocesado'] = dataset['tweet_preprocesado'].apply(convertir_emojis)
    dataset['tweet_preprocesado'] = dataset['tweet_preprocesado'].apply(depurar_emojis)
    dataset['tweet_preprocesado'] = dataset['tweet_preprocesado'].apply(quitar_tilde)
    texto_a_minusculas(dataset)
    dataset['tweet_preprocesado'] = dataset['tweet_preprocesado'].apply(filtrar_stopwords) #tokenizar y filtrar stopwords
    dataset['tweet_preprocesado'] = dataset['tweet_preprocesado'].apply(lematizar_texto) 

In [14]:
limpiar_tweets(tweets_2019_df)

In [15]:
#Guardamos dataset preprocesado
tweets_2019_df.to_excel("Dataset_preprocesado_2019.xlsx", index=False)

### Extraer características del dataset

In [19]:
#Obtenemos el path de la carpeta principal para llamar a archivos guardados
import os
path = os.path.normpath(os.getcwd() + os.sep + os.pardir)

In [20]:
# cargamos el dataset preprocesado para la extraccion de caracteristicas
tweets_2019_df = pd.DataFrame(pd.read_excel("Dataset_preprocesado_2019.xlsx")) 

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
# Creamos un nuevo vector tfidf con el antiguo vocabulario guardado de los unigramas (usado en el mejor modelo)
transformer = TfidfTransformer()
#cargamos el vocabulario
loaded_vec = CountVectorizer(decode_error="replace",vocabulary=pickle.load(open(path+"/3.Extraccion_caracteristicas/vocabulary_Unigrama.pkl", "rb")))
#transformamos los nuevos datos
tfidf = transformer.fit_transform(loaded_vec.fit_transform(tweets_2019_df["tweet_preprocesado"]))
vector_tfdf=tfidf.toarray()

"tfidf" tendrá la misma longitud de características que los datos entrenados.

## Predicción del  dataset mediante el modelo guardado de RF

In [22]:
#Cargamos el modelo que ofreció mejor rendimiento
import joblib 
modelo_rf = joblib.load(path+'/modelo_RF_Unigram.pkl')

In [23]:
#Predecimos el sentimiento
sentiment = modelo_rf.predict(vector_tfdf)

In [25]:
#Guardamos el resultado de la predicción en un dataframe (columna "Sentimiento_Predicho")
prediccion_df=pd.DataFrame()
prediccion_df['Sentimiento_Predicho']=pd.DataFrame(sentiment)

In [27]:
#Unimos el dataset cargado con la columna de "sentimiento_predicho" para 
#    comprobar los tweets que fueron clasificados con cada clase (1 y 0)
depresion_2019_df = pd.concat([tweets_2019_df, prediccion_df], axis=1) #pd.concat([df1, s1], axis=1)

depresion_2019_df.sample(6)

id        date      time  \
496   1.201163e+18  2019-12-01  10:36:58   
1038  1.080955e+18  2019-01-03  17:31:28   
2003  1.181384e+18  2019-10-07  20:39:46   
1461  1.183525e+18  2019-10-13  18:29:25   
1251  1.196482e+18  2019-11-18  12:36:10   
1901  1.121061e+18  2019-04-24  09:37:16   

                                                  tweet  \
496                 Un arquero de ansiedad nada normal.   
1038  100 de cada 1 persona que conozco sufre de dep...   
2003  Fiel a las prácticas del neoliberalismo ahora ...   
1461  @HarryPutter_1 Y creo q tu eres parte o pagado...   
1251             Que desmotivada estoy en la u señores.   
1901  @elcomerciocom La violencia familiar o la pres...   

                                                    geo  \
496   -2.6189724258280527,-79.5099020664484,28.20628...   
1038  -2.6189724258280527,-79.5099020664484,28.20628...   
2003  -0.1983403079450697,-77.53522051504922,27.8055...   
1461  -2.2579465031735535,-79.76289615008257,28.1849...   
1251  -2.6189724258280527,-79.5099020664484,28.20628...   
1901  -0.1983403079450697,-77.53522051504922,27.8055...   

                                     tweet_preprocesado  Sentimiento_Predicho  
496           ['arquero', 'ansiedad', 'nada', 'normal']                     1  
1038  ['cada', 'persona', 'conocer', 'sufrir', 'depr...                     0  
2003  ['fiel', 'practica', 'neoliberalismo', 'ahora'...                     0  
1461  ['creer', 'parte', 'pagar', 'desesperado', 'de...                     0  
1251                      ['desmotivada', 'u', 'señor']                     1  
1901  ['violencia', 'familiar', 'presion', 'economic...                     0

In [28]:
#eliminamos los tweets que fueron clasificados como no depresivos (etiqueta 0)
df_filtrado = depresion2019_df[depresion2019_df['Sentimiento_Predicho'] == 1]
#Guardamos el dataset solo con los tweets clasificados como depresivos para uso posterior
df_filtrado.to_excel("Tweets_Depresivos_2019_Predichos.xlsx", index=False)